In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
from torchvision.datasets import FashionMNIST, MNIST, CIFAR10, SVHN
import torchvision
from torchvision import transforms
import torchvision.utils as vision_utils
import matplotlib.pyplot as plt
import random
import os
import time
import math

DEVICE = torch.device('cuda')

In [2]:
def switch_to_device(dataset,device=None):
    final_X, final_Y = [], []
    for x, y in dataset:
        final_X.append(x)
        final_Y.append(y)
    X = torch.stack(final_X)
    Y = torch.tensor(final_Y)
    if device is not None:
        X = X.to(device)
        Y = Y.to(device)
    return torch.utils.data.TensorDataset(X, Y)


def get_mnist_dl(batch_size_train=256, batch_size_eval=256, device=torch.device('cpu')):
    transform = transforms.Compose([transforms.ToTensor()])
    
    data_train = MNIST('./datasets', train=True, download=True, transform=transform)
    data_train = switch_to_device(data_train, device=device)
    data_train, data_valid = torch.utils.data.random_split(data_train, [55000,5000])
    
    data_test = MNIST('./datasets', train=False, download=True, transform=transform)
    data_test = switch_to_device(data_test, device=device)
    
    train_dl = DataLoader(data_train, batch_size=batch_size_train, shuffle=True)
    valid_dl = DataLoader(data_valid, batch_size=batch_size_eval, shuffle=False)
    test_dl = DataLoader(data_test, batch_size=batch_size_eval, shuffle=False)
    
    return train_dl, valid_dl, test_dl

In [3]:

class MLP_Net(nn.Module):

  def __init__(self, num_classes=10) -> None:
    super().__init__()
    self.flatten = nn.Flatten()
    self.fc1 = nn.Linear(28*28, 1024)
    self.Relu1 = nn.ReLU()
    self.fc2 = nn.Linear(1024, 1024)
    self.Relu2 = nn.ReLU()
    self.fc3 = nn.Linear(1024, num_classes)
    #self.softmax = nn.Softmax()


  def forward(self, x: torch.Tensor) -> torch.Tensor:
    x = self.flatten(x)
    x = self.Relu1(self.fc1(x))
    x = self.Relu2(self.fc2(x))
    x = self.fc3(x)
    
    return x


In [4]:
class CNN_Net(nn.Module):
    def __init__(self, num_classes=10) -> None:
        super().__init__()
        self.conv1 = nn.Conv2d(1, 26, kernel_size=5, stride=1, padding = 0)
        self.maxpooling1 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv2 = nn.Conv2d(26, 52, kernel_size=3, stride=1, padding = 0)

        self.conv3 = nn.Conv2d(52, 10, kernel_size=1, stride=1, padding=0)
        self.maxpooling3 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.fc_1 = nn.Linear(5*5*10, 1000)
        self.fc_2 = nn.Linear(1000, num_classes)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.maxpooling1(F.relu(self.conv1(x)))
        x = F.relu(self.conv2(x))
        x = self.maxpooling3(F.relu(self.conv3(x)))
        x = torch.flatten(x, start_dim=1)
        x = F.relu(self.fc_1(x))
        x = self.fc_2(x)
        return x

In [5]:
def print_stats(stats):

  fig, (ax1, ax2) = plt.subplots(1,2,figsize=(7,3), dpi=110)
  ax1.grid()
  ax2.grid()

  ax1.set_title("ERM loss")
  ax2.set_title("Valid Acc")
  
  ax1.set_xlabel("iterations")
  ax2.set_xlabel("iterations")

  itrs = [x[0] for x in stats['train-loss']]
  loss = [x[1] for x in stats['train-loss']]
  ax1.plot(itrs, loss)

  itrs = [x[0] for x in stats['valid-acc']]
  acc = [x[1] for x in stats['valid-acc']]
  ax2.plot(itrs, acc)

  ax1.set_ylim(0.0, 4.05)
  ax2.set_ylim(0.0, 1.05)


@torch.no_grad()
def get_acc(model, dl):
  model.eval()
  acc = []
  for X, y in dl:
    #acc.append((torch.sigmoid(model(X)) > 0.5) == y)
    acc.append(torch.argmax(model(X), dim=1) == y)
  acc = torch.cat(acc)
  acc = torch.sum(acc)/len(acc)
  model.train()
  return acc.item()


def run_experiment(model, opt, train_dl, valid_dl, test_dl, use_forward_grad=False, num_forward_grad=1, max_epochs=20):

  itr = -1
  stats = {'train-loss': [], 'valid-acc':[]}
  time_list = []

  for epoch in range(max_epochs):
    for x, y in train_dl:
        itr += 1
        layer_inputs = {}
        def hook(mod, input):
            layer_inputs[mod] = input[0]
        for module in model.modules():
            module.register_forward_pre_hook(hook)
        opt.zero_grad()
        start_2 = time.clock()
        print(torch.cuda.memory_allocated()/1024/1024)
        loss = F.cross_entropy(model(x), y)
        print(torch.cuda.memory_allocated()/1024/1024)
        loss.backward()
        print(torch.cuda.memory_allocated()/1024/1024)
        
        if use_forward_grad:
          with torch.no_grad():
            #v_list = []
            da = torch.zeros(num_forward_grad, 1).to(DEVICE)
            for p in model.parameters():
              g = p.grad.view(-1)
              v = torch.randn(num_forward_grad, len(g), device=DEVICE)
              da  = da + (v @ g).view(num_forward_grad,1)
              #v_list.append(v)
              g = ((v @ g).view(num_forward_grad,1) * v).mean(dim=0)
              p.grad = g.view(p.grad.shape)
        '''
        if use_forward_grad:
          with torch.no_grad():
            #v_list = []
            da = torch.zeros(num_forward_grad, 1).to(DEVICE)
            for p in model.parameters():
              g = p.grad.view(-1)
              v = torch.randn(num_forward_grad, len(g), device=DEVICE)
              source = torch.randn(len(g), device=DEVICE)
              gap = math.floor(len(g)/num_forward_grad)
              for i in range(num_forward_grad):
                if i<num_forward_grad-1:
                  v[i] = F.pad(source[i*gap:(i+1)*gap], pad = (i*gap, len(g)-(i+1)*gap))
                else:
                  v[i] = F.pad(source[i*gap:], pad = (i*gap, 0))
              da  = da + (v @ g).view(num_forward_grad,1)
              #v_list.append(v)
              g = ((v @ g).view(num_forward_grad,1) * v).mean(dim=0)
              p.grad = g.view(p.grad.shape)
 
        
        if use_forward_grad:
          with torch.no_grad():
            for name, module in model.named_modules():
              if isinstance(module, torch.nn.Linear):
                grad_w = module.weight.grad
                approx = using_inputs_project(grad_w, layer_inputs[module])
                module.weight.grad = approx
                grad_b = module.bias.grad.view(-1)
                v = torch.randn(num_forward_grad, len(grad_b), device=DEVICE)
                grad_b = ((v @ grad_b).view(num_forward_grad,1) * v).mean(dim=0)
                module.bias.grad = grad_b.view(module.bias.grad.shape)
        '''
        
        #nn.utils.clip_grad_norm_(model.parameters(), max_norm=0.25, norm_type=2)
        opt.step()
        print(torch.cuda.memory_allocated()/1024/1024)
        time_list.append(time.clock()-start_2)

        stats['train-loss'].append((itr, loss.item()))

        if itr % 20 == 0:
          valid_acc = get_acc(model, valid_dl)
          stats['valid-acc'].append((itr, valid_acc))
          s = f"{epoch}:{itr} [train] loss:{loss.item():.3f}, [valid] acc:{valid_acc:.3f}, time: {np.sum(time_list)/len(time_list)}"
          print(s)
          time_list = []

  test_acc = get_acc(model, test_dl)
  print(f"[test] acc:{test_acc:.3f}")
  return stats


In [6]:
def using_inputs_project(grad, input):
    noise = torch.randn(grad.shape[0], device=DEVICE)
    '''
    inpt = torch.mean(input, dim = 0).view(-1)[None, :]

    entry_from_batch = torch.randint(low=0, high=len(input), size=[])
    inpt = input[entry_from_batch].view(-1)[None, :]
    inpt /= inpt.norm()
    '''
    q,r = torch.qr(input.T)
    entry_from_batch = torch.randint(low=0, high=len(input), size=[])
    inpt = q.T[entry_from_batch].view(-1)[None, :]
    expanded_noise = noise[:, None] * inpt
    return expanded_noise * torch.sum(expanded_noise * grad)

In [7]:
train_dl, valid_dl, test_dl = get_mnist_dl(device=DEVICE)

model = CNN_Net().to(DEVICE)

for p in model.parameters():
    g = p.view(-1)
    v = torch.normal(mean = torch.full((1, len(g)), 0.), std = torch.full((1, len(g)), 0.1)).to(DEVICE)
    p.data = v.view(p.shape)

opt = torch.optim.SGD(model.parameters(), lr=1e-4)


stats = run_experiment(model, opt, train_dl, valid_dl, test_dl, use_forward_grad=False, num_forward_grad = 1, max_epochs=20)

print_stats(stats)

212.353515625
245.7314453125
238.96142578125
238.96142578125
0:0 [train] loss:2.467, [valid] acc:0.088, time: 2.3395713
213.40478515625
246.7822265625
238.96142578125
238.96142578125
213.40478515625
246.7822265625
238.96142578125
238.96142578125
213.40478515625
246.7822265625
238.96142578125
238.96142578125
213.40478515625
246.7822265625
238.96142578125
238.96142578125
213.40478515625
246.7822265625
238.96142578125
238.96142578125
213.40478515625
246.7822265625
238.96142578125
238.96142578125
213.40478515625
246.7822265625
238.96142578125
238.96142578125
213.40478515625
246.7822265625
238.96142578125
238.96142578125
213.40478515625
246.7822265625
238.96142578125
238.96142578125
213.40478515625
246.7822265625
238.96142578125
238.96142578125
213.40478515625
246.7822265625
238.96142578125
238.96142578125
213.40478515625
246.7822265625
238.96142578125
238.96142578125
213.40478515625
246.7822265625
238.96142578125
238.96142578125
213.40478515625
246.7822265625
238.96142578125
238.9614257812

KeyboardInterrupt: 